In [1]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import concatenate, add, Lambda
from keras.layers import LSTM
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from helpers import TrainingMonitor
from helpers import Utils

In [3]:
output_path = "../output/"

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
db = h5py.File("../input/datasets/UCF-101-hdf5/fixed-sequence-xception-imagenet-features-shuffle.hdf5")
x = db["images"][:]
y = db["labels"][:]

train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [6]:
x_train.shape[1:]

(20, 2048)

In [7]:
num_classes = y_test.shape[1]
num_classes

50

In [17]:
model = Sequential()
model.add(LSTM(1024, batch_input_shape=(1, x_train.shape[1], x_train.shape[2]), stateful=True))
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model = Sequential()
model.add(LSTM(1024, input_shape=(x_train.shape[1:])))
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
#filepath=output_path + "progress/inception-weights-{epoch:02d}-{val_acc:.4f}.hdf5"
print(os.getpid())
filepath=output_path + "progress/sequence-xception-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

3136


In [22]:
history = model.fit(x_train, y_train,
            batch_size=16,
            epochs=2,
            validation_data=(x_val, y_val),
            shuffle="batch",
            callbacks=callbacks)

Train on 874 samples, validate on 187 samples
Epoch 1/2
874/874 [==============================] - 5s 6ms/step - loss: 0.1021 - acc: 0.9800 - val_loss: 0.1011 - val_acc: 0.9800
Epoch 2/2
874/874 [==============================] - 4s 5ms/step - loss: 0.1012 - acc: 0.9800 - val_loss: 0.1005 - val_acc: 0.9800


In [23]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 98.00%; Val: 98.00%; Test: 98.00%
